In [76]:
import numpy as np
import pandas as pd
import sqlite3
import time
import os

# Create Data Converter when Importing File
### Decoy = 1
### Target = 0
### D / T    = 0.5

In [47]:
#Check Target-Decoy and change to T/F:
def TDcheck(protein_ls):
    protein_ls = protein_ls.split(';')
    if any('XXX' in s for s in protein_ls):
        if any('XXX' not in s for s in protein_ls):
            return 0.5
        else:
            return 1
    else:
        return 0
    

    

# Create a Database

In [78]:
Database = sqlite3.connect('./test.db')

In [56]:
PSMQthreshold =1000000

# (TXT) Reading File & Writing to SQL Database

In [99]:
ID = [i for i in os.listdir('./input1') if i[-4:] == '.txt']


for i in ID:
    readfile_s = time.time()
    file = pd.read_csv('./input1/%s'%i,sep='\t',header=0,
                     usecols=['Peptide sequence', 'Protein(s)','percolator svm-score','PSM q-value'],
                     converters={'Protein(s)':TDcheck})
    file = file.loc[file['PSM q-value'] < PSMQthreshold].loc[:,['Peptide sequence', 'Protein(s)','percolator svm-score']]
    readfile_e = time.time()
    
    tosql_s = time.time()
    pd.io.sql.to_sql(file,'data',con=Database,if_exists='append',index=False)
    tosql_e = time.time()
    
    print('[f%s] read:%.1f, saveSQL:%.1f'%(i,readfile_e-readfile_s,tosql_e-tosql_s))


C:\Users\chienhua\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  del sys.path[0]


[fOCb4_N1_refseq-peptable - 複製 - 複製.txt] read:0.0, saveSQL:0.1
[fOCb4_N1_refseq-peptable - 複製.txt] read:0.0, saveSQL:0.1


# Remain MAX(percolator svm-score)

In [64]:
# Remain MAX(percolator svm-score)
delete_s = time.time()
#sql_command = 'DELETE FROM data WHERE (`Peptide sequence`,`percolator svm-score`) NOT IN (select `Peptide sequence`,MAX(`percolator svm-score`) from data GROUP BY `Peptide sequence`)'
#Database.execute(sql_command)
# Delete same value
sql_command = 'DELETE FROM data WHERE ROWID NOT IN (SELECT MIN(ROWID) FROM data GROUP BY `Peptide sequence`)'
Database.execute(sql_command)


delete_e = time.time()

print('delete:%.1f sec.'%(delete_e-delete_s))

OperationalError: row value misused

In [104]:
sql_command = 'DELETE FROM data WHERE ROWID NOT IN (SELECT MIN(ROWID) FROM data GROUP BY `Peptide sequence`)'
Database.execute(sql_command)
pd.read_sql('SELECT * FROM data', Database)

,Peptide sequence,Protein(s),percolator svm-score
0,+229.163MTQIMFETFNTPAMYVAIQAVLSLYASGR,0,6.80
1,-17.027QGQIIYNWQGAQSTQDEVAASAILTAQLDEELGGTPVQSR,0,5.52


In [101]:
sql_command = 'SELECT *,ROWID FROM data GROUP BY `Peptide sequence`,`percolator svm-score`'
pd.read_sql(sql_command, Database)

,Peptide sequence,Protein(s),percolator svm-score,rowid
0,+229.163MTQIMFETFNTPAMYVAIQAVLSLYASGR,0,6.80,1
1,+229.163MTQIMFETFNTPAMYVAIQAVLSLYASGR,0,6.90,3
2,-17.027QGQIIYNWQGAQSTQDEVAASAILTAQLDEELGGTPVQSR,0,5.52,2


In [100]:
pd.read_sql('SELECT * FROM data', Database)

,Peptide sequence,Protein(s),percolator svm-score
0,+229.163MTQIMFETFNTPAMYVAIQAVLSLYASGR,0,6.80
1,-17.027QGQIIYNWQGAQSTQDEVAASAILTAQLDEELGGTPVQSR,0,5.52
2,+229.163MTQIMFETFNTPAMYVAIQAVLSLYASGR,0,6.90
3,-17.027QGQIIYNWQGAQSTQDEVAASAILTAQLDEELGGTPVQSR,0,5.52
4,+229.163MTQIMFETFNTPAMYVAIQAVLSLYASGR,0,6.80
5,-17.027QGQIIYNWQGAQSTQDEVAASAILTAQLDEELGGTPVQSR,0,5.52


In [34]:
22059*2

44118

# Select all data by descending `percolator svm-score`

In [60]:
'''
df0 = pd.read_sql('SELECT * FROM data ORDER BY `percolator svm-score` DESC', Database)
df0
'''

"\ndf0 = pd.read_sql('SELECT * FROM data ORDER BY `percolator svm-score` DESC', Database)\ndf0\n"

In [59]:
sql_command = 'SELECT `Peptide sequence`,`Protein(s)`,`percolator svm-score` FROM data WHERE (`Peptide sequence`,`percolator svm-score`) IN (select `Peptide sequence`,MAX(`percolator svm-score`) FROM data GROUP BY `Peptide sequence`) AND (`PSM q-value` < %s) ORDER BY `percolator svm-score` DESC'%threshold
df0 = pd.read_sql(sql_command, Database)
df0


,Peptide sequence,Protein(s),percolator svm-score
0,+229.163LASPSGSTSSGLEVVAPEGTSAPGGGPGTLDDSATIC+...,0.0,7.74287
1,+229.163ILTVEDHYYEGGIGEAVSSAVVGEPGITVTHLAVNR,0.0,7.68700
2,+229.163YDGMVGFSADETWDSAIDTWSPSEWQMAYEPQGGSGYD...,0.0,7.67900
3,+229.163AQAALQAVNSVQSGNLALAASAAAVDAGMAMAGQSPVLR,0.0,7.63900
4,+229.163TTSGYAGGLSSAYGGLTSPGLSYSLGSSFGSGAGSSSFSR,0.0,7.49200
...,...,...,...
270029,TGAGNPGVLITLPSFFTIIK+229.163,1.0,0.01600
270030,+229.163TRHTLEK+229.163,0.0,0.01600
270031,+229.163K+229.163PNFILLLK+229.163ALLLSSYGK+229...,0.0,0.01600
270032,+229.163TPVK+229.163FWFLK+229.163NYLSPTFK+229.163,0.0,0.01600


In [43]:
max(df0['PSM q-value'])

0.008999407485371704

# parameter calculation (D)

In [31]:
'''
D = Database.execute("select `Protein(s)`,COUNT(`Protein(s)`) FROM data GROUP BY `Protein(s)` ORDER BY `Protein(s)` DESC")
D = list(D)

if len(D) == 3:
    D = D[0][1]+D[1][1]
elif len(D) == 2:
    D = D[0][1]
    
print('D=%s'%D)
'''

D=4634


# [DataFrame] p-value(peptide) & q-value(peptide)

In [68]:
'''
df0 = pd.read_sql('SELECT * FROM data ORDER BY `percolator svm-score` DESC', Database)

p_value = []
q_value = []
d = 0
d_dict = {1.0:1,0.5:1,0.0:0}
svm_score_change = df0.at[0,'percolator svm-score'] + 1
t = 0
t_dict = {1.0:0,0.5:1,0.0:1}

start = time.time()
for i in range(len(df0)):
    k = -df0.at[i,'Protein(s)']
    if k != -0.5:
        k = k + 0.5
    svm_score = df0.at[i,'percolator svm-score']
    if svm_score < svm_score_change:
        d += sum([d_dict[j] for j in df0.loc[df0['percolator svm-score'] == svm_score,'Protein(s)']])
        t += sum([t_dict[j] for j in df0.loc[df0['percolator svm-score'] == svm_score,'Protein(s)']])
        svm_score_change = svm_score
    p_value.append((d+k)/D)
    q_value.append(d/t)
df0['p-value'] = p_value    
df0['q-value'] = q_value

end = time.time()
print('%.1f'%(end-start))
'''

9.8


# [Cursor] p-value(peptide) & q-value(peptide)

In [11]:
#資料庫長度
length = Database.execute("select COUNT(`Peptide sequence`) FROM data WHERE (`Peptide sequence`,`percolator svm-score`) IN (select `Peptide sequence`,MAX(`percolator svm-score`) from data GROUP BY `Peptide sequence`)")
length = list(length)[0][0]
print('Length of Data = %s'%length)

Length of Data = 272983


In [25]:
#建立cursor庫
c = Database.cursor()

#降序排列放到cursor庫
sql_command = 'SELECT `Peptide sequence`,`Protein(s)`,`percolator svm-score` FROM data WHERE (`Peptide sequence`,`percolator svm-score`) IN (select `Peptide sequence`,MAX(`percolator svm-score`) FROM data GROUP BY `Peptide sequence`) ORDER BY `percolator svm-score` DESC'
c.execute(sql_command)

#max(SVM-score)+1
svm_score_change = Database.execute('SELECT MAX(`percolator svm-score`) FROM data')
svm_score_change = list(svm_score_change)[0][0]


#暫存
Peptide = []
d_list = []
t_list = []
k_list = []
#p_value = []
#q_value = []
d = 0
t = 0
d_dict = {1.0:1,0.5:1,0.0:0}
k_dict = {1.0:-0.5,0.5:-0.5,0.0:0.5}
t_dict = {1.0:0,0.5:1,0.0:1}


#建立txt檔案
txt = open('./test.csv','w')
txt.write('Peptide sequence,p-value,q-value\n')

#計算D
D = Database.execute("select `Protein(s)`,COUNT(`Protein(s)`) FROM data WHERE (`Peptide sequence`,`percolator svm-score`) IN (select `Peptide sequence`,MAX(`percolator svm-score`) from data GROUP BY `Peptide sequence`) GROUP BY `Protein(s)` ORDER BY `Protein(s)` DESC")
D = list(D)
if len(D) == 3:
    D = D[0][1]+D[1][1]
elif len(D) == 2:
    D = D[0][1]
    
#開始迴圈計算
start = time.time()
for i in range(length):
    peptide,protein,score = c.fetchone()
    if score == svm_score_change:
        Peptide.append(peptide)
        d_list.append(d_dict[protein])
        t_list.append(t_dict[protein])
        k_list.append(k_dict[protein])
    else:
        #計算
        d += sum(d_list)
        t += sum(t_list)
        rep_number = len(Peptide)
        p_value = (np.repeat(d,rep_number) + np.array(k_list)) / D
        if t != 0:
            q_value = np.repeat(d/t,rep_number)
        else:
            q_value = np.repeat('',rep_number)
        
        #寫檔
        w = ''.join([m+','+str(n)+','+str(o)+'\n' for m,n,o in zip(Peptide,p_value,q_value)])
        txt.write(w)
        
        #清除list
        Peptide = [peptide]
        d_list = [d_dict[protein]]
        t_list = [t_dict[protein]]
        k_list = [k_dict[protein]]
        svm_score_change = score
    #計算最後一個
    if i == length-1:
        #計算
        d += sum(d_list)
        t += sum(t_list)
        rep_number = len(Peptide)
        p_value = (np.repeat(d,rep_number) + np.array(k_list)) / D
        q_value = np.repeat(d/t,rep_number)
        
        #寫檔
        w = ''.join([m+','+str(n)+','+str(o)+'\n' for m,n,o in zip(Peptide,p_value,q_value)])
        txt.write(w)
        txt.close()
end = time.time()
print('time-used:%.1f sec.'%(end-start))

time-used:1.9 sec.


In [77]:
Database.close()

In [9]:
str(x)

''

In [12]:
f1 = pd.read_csv('./input/%s'%ID[0],sep='\t',header=0,
                 usecols=['Peptide sequence', 'Protein(s)','percolator svm-score'],
                 converters={'Protein(s)':TDcheck})

In [13]:
f2 = pd.read_csv('./input/%s'%ID[1],sep='\t',header=0,
                 usecols=['Peptide sequence', 'Protein(s)','percolator svm-score'],
                 converters={'Protein(s)':TDcheck})

In [18]:
f2

,Peptide sequence,Protein(s),percolator svm-score
0,+42.011M+15.995DGFAGSLDDSISAASTSDVQDR,0.0,4.837
1,+42.011MLTDSGGGGTSFEEDLDSVAPR,0.0,4.566
2,-17.027QGQIIYNWQGAQSTQDEVAASAILTAQLDEELGGTPVQSR,0.0,4.973
3,+42.011MDGFAGSLDDSISAASTSDVQDR,0.0,4.497
4,-17.027QITAQAWDGTTDYQVEETSR,0.0,4.163
...,...,...,...
221117,+229.163HLVGVC+45.988YTEDEAK+229.163ELAAEVEVQD...,0.0,0.307
221118,+229.163HNK+229.163ELQSMLK+229.163PGGQVDGDEALE...,0.0,0.133
221119,+229.163NRPSVPPPPQPPGVHSAGDSSLTNTAPTASK+229.163,0.0,0.272
221120,+229.163APLNVTNTAGTSLPSVDLLQK+229.163LRLTPQHDQ...,0.0,0.049


In [22]:
len(set(list(f1['Peptide sequence']) +list(f2['Peptide sequence'])))

272983

In [7]:
x = '+24219.2BCKAKDRO+23.233FDHDL'
''.join([i for i in x if i.isalpha()])

'BCKAKDROFDHDL'

In [ ]:
#未來可做
APPEND一個完，選取最大，刪掉原本的DB，創立新的DB。

In [6]:
[i for i in x if i.isalpha]

['+',
 '2',
 '4',
 '2',
 '1',
 '9',
 '.',
 '2',
 'B',
 'C',
 'K',
 'A',
 'K',
 'D',
 'R',
 'O',
 '+',
 '2',
 '3',
 '.',
 '2',
 '3',
 '3',
 'F',
 'D',
 'H',
 'D',
 'L']

In [ ]:
x = '+'
x.isapl